In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import time
import copy

<h3>1. Load training images</h3>

In [ ]:
train_images = torch.tensor(np.load('../input/mais-202-winter-2022/train_images.npy').astype('uint8'))
df = pd.read_csv('../input/mais-202-winter-2022/train_labels.csv')
labels = torch.tensor(df['label'])

<h3>2. Visualize images </h3>

In [ ]:
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
def show_image(arr, title):
    two_d = np.reshape(arr, (28, 28)) * 255
    plt.imshow(two_d, interpolation='nearest', cmap='gray')
    plt.title(label_names[title])
    plt.show()

for i in range(5):
    show_image(train_images[i], labels[i])

<h3>3. Define Dataset and Dataloaders</h3>

In [ ]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.RandomCrop(28, padding=2),
    transforms.RandomHorizontalFlip(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081)),
    transforms.RandomErasing(p=0.2)
])

test_transforms = transforms.Compose([
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))
])

In [ ]:
from torch.utils.data import Dataset
import cv2

class FashionMNIST(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.labels = labels
        self.transforms = transforms;
        self.images = torch.full((images.shape[0], 3, 28, 28), 0.0)
        for index, image in enumerate(images):
            self.images[index] = torch.Tensor(np.moveaxis(cv2.cvtColor(image.numpy(), cv2.COLOR_GRAY2RGB), -1, 0))
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        if self.transforms is not None:
            image = self.transforms(image)
        if self.labels is not None:
            label = self.labels[index]
            return (image, label)
        return image

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_images, labels, test_size=0.2)

train_ds = FashionMNIST(X_train, y_train, transforms=train_transforms)
valid_ds = FashionMNIST(X_valid, y_valid, transforms=test_transforms)

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=32, shuffle=False)

In [ ]:
image_datasets = {
    'train': train_ds,
    'val': valid_ds
}

dataloaders = {
    'train': train_dl,
    'val': valid_dl
}

dataset_sizes = {
    'train': len(train_ds),
    'val': len(valid_ds)
}

<h3>4. Define Model</h3>

In [ ]:
pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=10)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
def train(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_weights = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            
            for images, labels in dataloaders[phase]:
                images = images.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_accuracy = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Accuracy: {:.4f}'.format(phase.capitalize(), epoch_loss, epoch_accuracy))

            if phase == 'val' and epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                best_weights = copy.deepcopy(model.state_dict())
                
        scheduler.step()
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best validation accuracy: {:4f}'.format(best_accuracy))

    model.load_state_dict(best_weights)
    return model

<h3>5. Train Model</h3>

In [ ]:
model = train(model, criterion, optimizer, scheduler, num_epochs=100)

In [ ]:
torch.save(model.state_dict(), 'state.pt')

<h3>6. Inference</h3>

In [ ]:
test = torch.tensor(np.load('../input/mais-202-winter-2022/test_images.npy').astype('uint8'))

In [ ]:
test_ds = FashionMNIST(test, transforms=test_transforms)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=False)

In [ ]:
def predict():
    model.eval()
    y_pred = None
    for i, images in enumerate(test_dl):
        with torch.no_grad():
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            if (y_pred is None):
                y_pred = preds
            else:
                y_pred = torch.cat((y_pred, preds))
    return y_pred.cpu().numpy()

In [ ]:
y_pred = predict()

In [ ]:
df = pd.read_csv('../input/mais-202-winter-2022/sample_submission.csv')
df['label'] = y_pred
df.to_csv('submission.csv', index=False)